In [1]:
# !pip install memory-profiler

In [2]:
# !pip install --upgrade ipython

In [3]:
# !pip install pyarrow

In [4]:
import numpy as np

In [5]:
import IPython

In [6]:
%load_ext memory_profiler

In [7]:
import pandas as pd

In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc

# House keeping information

In [9]:
import psutil

# Get the current process ID
process_id = os.getpid()

# Get the memory information
memory_info = psutil.Process(process_id).memory_info()

print(f"Memory used: {memory_info.rss / 1024 / 1024:.2f} MB")

Memory used: 197.26 MB


In [10]:
current_directory = os.getcwd()
current_directory

'/home/hermonpe/SIADS699/notebooks'

In [11]:
# Navigate to the parent directory
parent_directory = os.path.dirname(current_directory)
parent_directory

'/home/hermonpe/SIADS699'

# Read Data Files

## rows-genes.csv
	Contains information about the genes for which fpkm values were calculated. 

	gene_id
		Unique identifier for the gene.

	chromosome
		Chromosome associated with the gene.

	gene_entrez_id, gene_symbol, gene_name
		entrez_id, NCBI symbol, and name of the gene.

In [12]:
%memit
rows_genes = pd.read_csv("../data/external/rows-genes.csv")
rows_genes

peak memory: 197.34 MiB, increment: 0.00 MiB


,gene_id,chromosome,gene_entrez_id,gene_symbol,gene_name
0,499304660,1,100287102,DDX11L1,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...
1,499304661,1,653635,WASH7P,WAS protein family homolog 7 pseudogene
2,499304662,1,102466751,MIR6859-1,microRNA 6859-1
3,499304663,1,100302278,MIR1302-2,microRNA 1302-2
4,499304664,1,645520,FAM138A,"family with sequence similarity 138, member A"
...,...,...,...,...,...
50276,499355059,MT,4541,ND6,NADH dehydrogenase subunit 6
50277,499355060,MT,4556,TRNE,tRNA-Glu
50278,499355061,MT,4519,CYTB,cytochrome b
50279,499355062,MT,4576,TRNT,tRNA-Thr


## donorInformation.csv

In [13]:
%memit
donorinformation = pd.read_csv("../data/external/donorInformation.csv")
donorinformation

peak memory: 214.72 MiB, increment: 0.00 MiB


,donor_id,name,age,sex,apo_e4_allele,education_years,age_at_first_tbi,longest_loc_duration,cerad,num_tbi_w_loc,dsm_iv_clinical_diagnosis,control_set,nincds_arda_diagnosis,ever_tbi_w_loc,race,hispanic,act_demented,braak,nia_reagan
0,326765665,H14.09.078,87,M,N,16,0,Unknown or N/A,0,0,No Dementia,31,No Dementia,N,White,Not Hispanic,No Dementia,1,1
1,326765656,H14.09.069,95-99,M,N,17,12,1-2 min,2,1,No Dementia,26,No Dementia,Y,White,Not Hispanic,No Dementia,5,2
2,326765654,H14.09.067,85,M,Y,10,72,< 10 sec,3,1,Vascular,25,"Dementia, Type Unknown",Y,White,Not Hispanic,Dementia,4,2
3,467056391,H15.09.103,90-94,F,N,11,87,< 10 sec,0,1,No Dementia,52,No Dementia,Y,White,Not Hispanic,No Dementia,4,0
4,309335447,H14.09.010,100+,M,Y,16,0,Unknown or N/A,3,0,Alzheimer's Disease Type,28,Possible Alzheimer'S Disease,N,White,Not Hispanic,Dementia,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,309335458,H14.09.021,88,M,N,12,18,> 1 hr,0,1,Alzheimer's Disease Type,21,Probable Alzheimer'S Disease,Y,White,Not Hispanic,Dementia,1,1
103,309335462,H14.09.025,84,M,N,16,23,10 min - 1 hr,0,2,Alzheimer's Disease Type,40,Probable Alzheimer'S Disease,Y,White,Not Hispanic,Dementia,0,0
104,309335454,H14.09.017,79,M,N,16,13,< 10 sec,2,3,No Dementia,44,No Dementia,Y,White,Not Hispanic,No Dementia,3,2
105,309335486,H14.09.049,95-99,M,N,21,18,3-5 min,1,1,No Dementia,12,No Dementia,Y,White,Not Hispanic,No Dementia,3,1


## columns-samples.csv

columns-samples.csv 
	Contains information about the samples profiled with RNA sequencing
	
	rnaseq_profile_id
		Expression profile obtained from aligning the RNA-Seq data to the GRCh38.p2 reference genome.
	
	donor_id and donor_name
		Donor from which the sample was dissected
		
	specimen_id and specimen_name
		Specimen from which the sample was dissected (i.e., a particular brain structure from a particular donor)
		
	rna_well_id
		Unique identifier of the sample.

	polygon_id
		Unique identifier of an avg_graphic_object that outlines where the sample was cut from.

	structure_id, structure_abbreviation, structure_color, structure_name
		Label that groups samples by brain region (hippocampus, temporal cortex, parietal cortex, and forebrain white matter).

	hemisphere
		Hemisphere from which the processed sample was collected


In [14]:
%memit
columns_samples = pd.read_csv("../data/external/columns-samples.csv")
columns_samples

peak memory: 214.92 MiB, increment: 0.00 MiB


,rnaseq_profile_id,donor_id,donor_name,specimen_id,specimen_name,rna_well_id,polygon_id,structure_id,structure_acronym,structure_color,structure_name,hemisphere
0,488395315,309335467,H14.09.030,309357843,H14.09.030.TCx.01,395325172,320817998,10235,TCx,#ebbfd0,temporal neocortex,left
1,496100277,309335441,H14.09.004,309357624,H14.09.004.PCx.01,320630866,310967169,10557,FWM,#f2f1f0,white matter of forebrain,right
2,496100278,309335438,H14.09.001,309357596,H14.09.001.PCx.01,320630834,310790571,10557,FWM,#f2f1f0,white matter of forebrain,left
3,496100279,309335438,H14.09.001,309357599,H14.09.001.TCx.01,320630838,310790522,10235,TCx,#ebbfd0,temporal neocortex,left
4,496100281,309335439,H14.09.002,309357603,H14.09.002.HIP.01,320630842,310790372,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),right
...,...,...,...,...,...,...,...,...,...,...,...,...
372,496100667,467056391,H15.09.103,467179071,H15.09.103.TCx.01,482655826,480366830,10235,TCx,#ebbfd0,temporal neocortex,right
373,496100669,467056391,H15.09.103,467179068,H15.09.103.PCx.01,482655822,480363830,10557,FWM,#f2f1f0,white matter of forebrain,right
374,496100670,467056406,H15.09.107,467179104,H15.09.107.TCx.01,482655780,480363840,10235,TCx,#ebbfd0,temporal neocortex,right
375,496100671,467056391,H15.09.103,467179065,H15.09.103.HIP.01,482655820,480366825,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),right


## fpkm_table_normalized.csv

fpkm_table_normalized.csv
	Contains the (row, column) matrix of fpkm values obtained for each (gene, sample) after correcting for RIN and batch effects. These are the data displayed in the RNA-Seq page heatmap
	The first row contains the unique identifiers of the RNA-seq profiles of the samples (rnaseq_profile_id)
	The first column contains the gene unique identifiers (gene_id)

In [16]:
# uncoment the cell below if you need to unzip the fpkm_table_normalized file

# import zipfile
# # Specify the path to your zip file
# zip_file_path = '../data/external/fpkm_table_normalized.csv.zip'

# # Specify the extraction directory
# extracted_folder = '../data/external'

# # Open the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     # Extract all the contents into the specified directory
#     zip_ref.extractall(extracted_folder)
    


In [17]:
%memit
fpkm_normalized = pd.read_csv("../data/external/fpkm_table_normalized.csv")
fpkm_normalized


peak memory: 215.04 MiB, increment: 0.00 MiB


,gene_id \ rnaseq_profile_id,488395315,496100277,496100278,496100279,496100281,496100283,496100284,496100285,496100287,...,496100661,496100663,496100664,496100665,496100666,496100667,496100669,496100670,496100671,496100672
0,499304660,0.655725,0.095143,0.000000,0.000000,0.000000,0.000000,0.000000,0.151241,0.172735,...,0.280921,0.464819,0.000000,0.316875,0.250307,0.000000,0.000000,0.126133,0.000000,0.000000
1,499304661,4.526404,8.855850,4.868456,4.851842,3.600344,5.130615,5.496470,4.830538,4.565007,...,5.996864,3.933522,4.831699,4.727912,4.645094,5.664388,8.329654,5.712798,4.639818,4.708020
2,499304662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,499304663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,499304664,0.039654,0.016492,0.000000,0.170431,0.000000,0.007901,0.137523,0.087973,0.000000,...,0.000000,0.073427,0.040766,0.044743,0.119941,0.114818,0.143532,0.000000,0.075834,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50276,499355059,59.293828,24.446436,37.608601,50.775653,43.398201,60.780414,47.585662,39.506123,34.014895,...,65.033322,53.263396,48.865464,78.002569,53.859899,70.488572,49.578196,58.632013,72.621158,41.339535
50277,499355060,0.000000,0.000000,3.323079,0.000000,3.422902,3.618383,0.000000,7.195433,0.000000,...,1.960015,0.000000,0.000000,2.878588,2.771940,0.000000,0.000000,4.271477,10.262135,1.962493
50278,499355061,409.916660,375.295129,358.011161,364.972710,259.190920,476.317345,284.561050,300.710856,369.302275,...,525.768569,525.173676,484.113894,771.495569,433.073844,554.106124,460.322065,482.301655,592.635518,383.616738
50279,499355062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Data Transformation

In [18]:
%memit
# lets change the first column name to gene_id and make it the index
fpkm_normalized.columns = ['gene_id'] + list(fpkm_normalized.columns[1:])
fpkm_normalized.set_index('gene_id', inplace=True)
fpkm_normalized.head()

peak memory: 481.33 MiB, increment: 0.00 MiB


,488395315,496100277,496100278,496100279,496100281,496100283,496100284,496100285,496100287,496100288,...,496100661,496100663,496100664,496100665,496100666,496100667,496100669,496100670,496100671,496100672
gene_id,,,,,,,,,,,,,,,,,,,,,
499304660,0.655725,0.095143,0.000000,0.000000,0.000000,0.000000,0.000000,0.151241,0.172735,0.000000,...,0.280921,0.464819,0.000000,0.316875,0.250307,0.000000,0.000000,0.126133,0.000000,0.00000
499304661,4.526404,8.855850,4.868456,4.851842,3.600344,5.130615,5.496470,4.830538,4.565007,5.638256,...,5.996864,3.933522,4.831699,4.727912,4.645094,5.664388,8.329654,5.712798,4.639818,4.70802
499304662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
499304663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
499304664,0.039654,0.016492,0.000000,0.170431,0.000000,0.007901,0.137523,0.087973,0.000000,0.166256,...,0.000000,0.073427,0.040766,0.044743,0.119941,0.114818,0.143532,0.000000,0.075834,0.00000


In [19]:
%memit
# on the fpkm_normalized each row is a gene and each column is a profile
# First lets transpose the taple
# check the shape of the original fpkm_normalized data
fpkm_normalized.shape

peak memory: 481.43 MiB, increment: 0.00 MiB


(50281, 377)

In [20]:
%memit
transposed_fpkm = fpkm_normalized.T
transposed_fpkm.head()

peak memory: 481.43 MiB, increment: 0.00 MiB


gene_id,499304660,499304661,499304662,499304663,499304664,499304665,499304666,499304667,499304668,499304669,...,499355054,499355055,499355056,499355057,499355058,499355059,499355060,499355061,499355062,499355063
488395315,0.655725,4.526404,0.0,0.0,0.039654,0.0,0.0,0.0,0.0,0.317608,...,406.688243,0.0,0.0,0.0,177.049256,59.293828,0.000000,409.916660,0.0,0.000000
496100277,0.095143,8.855850,0.0,0.0,0.016492,0.0,0.0,0.0,0.0,0.955393,...,415.503112,0.0,0.0,0.0,138.242472,24.446436,0.000000,375.295129,0.0,0.000000
496100278,0.000000,4.868456,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.361740,...,524.577960,0.0,0.0,0.0,171.511364,37.608601,3.323079,358.011161,0.0,0.000000
496100279,0.000000,4.851842,0.0,0.0,0.170431,0.0,0.0,0.0,0.0,1.357990,...,428.282485,0.0,0.0,0.0,176.226091,50.775653,0.000000,364.972710,0.0,1.255010
496100281,0.000000,3.600344,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.005507,...,367.205701,0.0,0.0,0.0,115.521316,43.398201,3.422902,259.190920,0.0,6.467808


In [21]:
transposed_fpkm = transposed_fpkm.rename_axis("rnaseq_profile_id")

In [22]:
transposed_fpkm.head()

gene_id,499304660,499304661,499304662,499304663,499304664,499304665,499304666,499304667,499304668,499304669,...,499355054,499355055,499355056,499355057,499355058,499355059,499355060,499355061,499355062,499355063
rnaseq_profile_id,,,,,,,,,,,,,,,,,,,,,
488395315,0.655725,4.526404,0.0,0.0,0.039654,0.0,0.0,0.0,0.0,0.317608,...,406.688243,0.0,0.0,0.0,177.049256,59.293828,0.000000,409.916660,0.0,0.000000
496100277,0.095143,8.855850,0.0,0.0,0.016492,0.0,0.0,0.0,0.0,0.955393,...,415.503112,0.0,0.0,0.0,138.242472,24.446436,0.000000,375.295129,0.0,0.000000
496100278,0.000000,4.868456,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.361740,...,524.577960,0.0,0.0,0.0,171.511364,37.608601,3.323079,358.011161,0.0,0.000000
496100279,0.000000,4.851842,0.0,0.0,0.170431,0.0,0.0,0.0,0.0,1.357990,...,428.282485,0.0,0.0,0.0,176.226091,50.775653,0.000000,364.972710,0.0,1.255010
496100281,0.000000,3.600344,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.005507,...,367.205701,0.0,0.0,0.0,115.521316,43.398201,3.422902,259.190920,0.0,6.467808


In [23]:
transposed_fpkm = fpkm_normalized.T
# Set the name of the first column to "gene_id"
transposed_fpkm.columns.name = "rnaseq_profile_id"

In [24]:
transposed_fpkm.index = transposed_fpkm.index.astype(int)

In [25]:
transposed_fpkm.head()

rnaseq_profile_id,499304660,499304661,499304662,499304663,499304664,499304665,499304666,499304667,499304668,499304669,...,499355054,499355055,499355056,499355057,499355058,499355059,499355060,499355061,499355062,499355063
488395315,0.655725,4.526404,0.0,0.0,0.039654,0.0,0.0,0.0,0.0,0.317608,...,406.688243,0.0,0.0,0.0,177.049256,59.293828,0.000000,409.916660,0.0,0.000000
496100277,0.095143,8.855850,0.0,0.0,0.016492,0.0,0.0,0.0,0.0,0.955393,...,415.503112,0.0,0.0,0.0,138.242472,24.446436,0.000000,375.295129,0.0,0.000000
496100278,0.000000,4.868456,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.361740,...,524.577960,0.0,0.0,0.0,171.511364,37.608601,3.323079,358.011161,0.0,0.000000
496100279,0.000000,4.851842,0.0,0.0,0.170431,0.0,0.0,0.0,0.0,1.357990,...,428.282485,0.0,0.0,0.0,176.226091,50.775653,0.000000,364.972710,0.0,1.255010
496100281,0.000000,3.600344,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.005507,...,367.205701,0.0,0.0,0.0,115.521316,43.398201,3.422902,259.190920,0.0,6.467808


### Combine donor and sample information

In [26]:
%memit
# merge donorinformation and columns_samples
sample_merged_df = pd.merge(columns_samples, donorinformation,on='donor_id')
sample_merged_df.set_index('rnaseq_profile_id', inplace=True)
sample_merged_df.head()

peak memory: 626.18 MiB, increment: 0.00 MiB


,donor_id,donor_name,specimen_id,specimen_name,rna_well_id,polygon_id,structure_id,structure_acronym,structure_color,structure_name,...,num_tbi_w_loc,dsm_iv_clinical_diagnosis,control_set,nincds_arda_diagnosis,ever_tbi_w_loc,race,hispanic,act_demented,braak,nia_reagan
rnaseq_profile_id,,,,,,,,,,,,,,,,,,,,,
488395315,309335467,H14.09.030,309357843,H14.09.030.TCx.01,395325172,320817998,10235,TCx,#ebbfd0,temporal neocortex,...,0,No Dementia,3,No Dementia,N,White,Not Hispanic,No Dementia,3,2
496100352,309335467,H14.09.030,309357840,H14.09.030.PCx.01,395325168,320965916,10557,FWM,#f2f1f0,white matter of forebrain,...,0,No Dementia,3,No Dementia,N,White,Not Hispanic,No Dementia,3,2
496100381,309335467,H14.09.030,309357837,H14.09.030.HIP.01,395325166,320817993,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),...,0,No Dementia,3,No Dementia,N,White,Not Hispanic,No Dementia,3,2
496100386,309335467,H14.09.030,309357840,H14.09.030.PCx.01,395325170,320965914,10208,PCx,#f0dabe,parietal neocortex,...,0,No Dementia,3,No Dementia,N,White,Not Hispanic,No Dementia,3,2
496100277,309335441,H14.09.004,309357624,H14.09.004.PCx.01,320630866,310967169,10557,FWM,#f2f1f0,white matter of forebrain,...,0,No Dementia,14,No Dementia,N,White,Not Hispanic,No Dementia,3,1


In [27]:
sample_merged_df.shape

(377, 29)

### Assing a donor to the rna profile

In [28]:
%memit
merged_df =  pd.merge(transposed_fpkm, sample_merged_df, left_index=True, right_index=True)
merged_df = merged_df.rename_axis("rnaseq_profile_id")

peak memory: 627.03 MiB, increment: 0.00 MiB


In [29]:
merged_df.head()

,499304660,499304661,499304662,499304663,499304664,499304665,499304666,499304667,499304668,499304669,...,num_tbi_w_loc,dsm_iv_clinical_diagnosis,control_set,nincds_arda_diagnosis,ever_tbi_w_loc,race,hispanic,act_demented,braak,nia_reagan
rnaseq_profile_id,,,,,,,,,,,,,,,,,,,,,
488395315,0.655725,4.526404,0.0,0.0,0.039654,0.0,0.0,0.0,0.0,0.317608,...,0,No Dementia,3,No Dementia,N,White,Not Hispanic,No Dementia,3,2
496100277,0.095143,8.855850,0.0,0.0,0.016492,0.0,0.0,0.0,0.0,0.955393,...,0,No Dementia,14,No Dementia,N,White,Not Hispanic,No Dementia,3,1
496100278,0.000000,4.868456,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.361740,...,2,No Dementia,2,No Dementia,Y,White,Not Hispanic,No Dementia,6,2
496100279,0.000000,4.851842,0.0,0.0,0.170431,0.0,0.0,0.0,0.0,1.357990,...,2,No Dementia,2,No Dementia,Y,White,Not Hispanic,No Dementia,6,2
496100281,0.000000,3.600344,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.005507,...,0,Alzheimer's Disease Type,2,Probable Alzheimer'S Disease,N,White,Not Hispanic,Dementia,1,1


#### Export data

In [30]:
# Export data as parque
merged_df.columns = merged_df.columns.astype(str)
merged_df.to_parquet("../data/processed/merged_df.parquet", engine='pyarrow')

<font color="black" size="15">END OF NOTEBOOK</font>